In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, TimeDistributed, ConvLSTM2D, Reshape
import tensorflow as tf
import sklearn.metrics as sm
import keras
from keras.losses import huber_loss


from keras import backend as K

Using TensorFlow backend.


In [2]:
data = pd.read_csv("train.csv")

In [3]:
dtest = pd.read_csv("test.csv")

In [4]:
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data =  data.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [6]:
data.head()

,y,X10,X11,X12,X13,X14,X15,X16,X17,X18,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,88.53,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,76.26,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.shape

(4209, 369)

In [8]:
dtest

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,8410,aj,h,as,f,d,aa,j,e,0,...,0,0,0,0,0,0,0,0,0,0
4205,8411,t,aa,ai,d,d,aa,j,y,0,...,0,1,0,0,0,0,0,0,0,0
4206,8413,y,v,as,f,d,aa,d,w,0,...,0,0,0,0,0,0,0,0,0,0
4207,8414,ak,v,as,a,d,aa,c,q,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
dtest =  dtest.drop(["ID","X0","X1","X2","X3","X4","X5","X6","X8" ], axis = 1)

In [10]:
dtest.head() 

,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [11]:
dtest.shape

(4209, 368)

In [12]:
X_train, y_train = data.values[0:3499,1:], data.values[0:3499, :1].ravel()
X_valid, y_valid = data.values[3500:4208,1:], data.values[3500:4208, :1].ravel()
X_test = dtest.values[:, :]

In [13]:
print("X_train----",X_train.shape)
print("y_train----",y_train.shape)      
print("X_valid----",X_valid.shape)
print("y_valid----",y_valid.shape)
print("X_test-----",X_test.shape)

X_train---- (3499, 368)
y_train---- (3499,)
X_valid---- (708, 368)
y_valid---- (708,)
X_test----- (4209, 368)


In [14]:
X_train[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       1., 0., 0., 0., 0.

In [15]:
y_train[0]

130.81

In [16]:
# y_train = min_max_scaler.fit_transform(y_train)


In [17]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
def metrics(pred, y_test):
    evs = sm.explained_variance_score(y_test, pred)
    me = sm.max_error(y_test, pred)
    mae = sm.mean_absolute_error(y_test, pred)
    mse = sm.mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    #msle = sm.mean_squared_log_error(y_test, pred)
    m_ae = sm.median_absolute_error(y_test, pred)
    r2 = sm.r2_score(y_test, pred)
    #mpd = sm.mean_poisson_deviance(y_test, pred)
    #mgd = sm.mean_gamma_deviance(y_test, pred)
    mape = mean_absolute_percentage_error(pred, y_test)
    return({'Explained Variance Score': evs,
            'Max Error': me,
            'Mean Absolute Error': mae,
            'Mean Squared Error': mse,
            'Root Mean Squared Error': rmse,
            #'Mean Squared Log Error': msle,
            'Median Absolute Error': m_ae,
            'R² Score': r2,
            #'Mean Poisson Deviance': mpd,
            #'Mean Gamma Deviance': mgd,
            'Mean Absolute Percentage Error': mape
            })

In [19]:
subsequences = 16
timesteps = X_train.shape[1]//subsequences

In [20]:
timesteps 

23

In [21]:
X_train = X_train.reshape((X_train.shape[0], subsequences, 1, timesteps, 1))
X_valid = X_valid.reshape((X_valid.shape[0], subsequences, 1, timesteps, 1))
X_test  = X_test.reshape((X_test.shape[0], subsequences, 1, timesteps, 1))

In [22]:
X_train.shape

(3499, 16, 1, 23, 1)

In [23]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=50)

In [24]:
modelConvLSTM = Sequential()
modelConvLSTM.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', return_sequences=True,input_shape=(X_train.shape[1], 1, X_train.shape[3], X_train.shape[4])))
modelConvLSTM.add(ConvLSTM2D(filters=32, kernel_size=(1,2), activation='relu'))
modelConvLSTM.add(Flatten())
modelConvLSTM.add(Dense(64))
modelConvLSTM.add(Dense(32))
modelConvLSTM.add(Dense(1,kernel_regularizer=tf.keras.regularizers.l2(0.01),activation
             ='linear'))
modelConvLSTM.compile(optimizer='adam', loss="huber_loss")
history = modelConvLSTM.fit(X_train, y_train, batch_size=512, epochs=700, verbose=2, callbacks=[early_stop], validation_data = (X_valid, y_valid))

Train on 3499 samples, validate on 708 samples
Epoch 1/700
 - 44s - loss: 100.1434 - val_loss: 97.1109
Epoch 2/700
 - 21s - loss: 94.4823 - val_loss: 74.1245
Epoch 3/700
 - 22s - loss: 44.2052 - val_loss: 37.4684
Epoch 4/700
 - 21s - loss: 29.6594 - val_loss: 27.3844
Epoch 5/700
 - 21s - loss: 18.7973 - val_loss: 10.5793
Epoch 6/700
 - 21s - loss: 13.9387 - val_loss: 13.1437
Epoch 7/700
 - 21s - loss: 12.3776 - val_loss: 10.2387
Epoch 8/700
 - 21s - loss: 11.1250 - val_loss: 10.0782
Epoch 9/700
 - 21s - loss: 10.5501 - val_loss: 9.9557
Epoch 10/700
 - 21s - loss: 10.3497 - val_loss: 9.4309
Epoch 11/700
 - 22s - loss: 10.1640 - val_loss: 9.3498
Epoch 12/700
 - 23s - loss: 10.1533 - val_loss: 9.2551
Epoch 13/700
 - 22s - loss: 10.1229 - val_loss: 9.2375
Epoch 14/700
 - 21s - loss: 9.9826 - val_loss: 9.6494
Epoch 15/700
 - 21s - loss: 9.9608 - val_loss: 9.1074
Epoch 16/700
 - 22s - loss: 9.8324 - val_loss: 9.0109
Epoch 17/700
 - 21s - loss: 9.7742 - val_loss: 8.9549
Epoch 18/700
 - 21s - 

Epoch 151/700
 - 46s - loss: 5.2065 - val_loss: 4.3536
Epoch 152/700
 - 45s - loss: 5.3595 - val_loss: 4.8363
Epoch 153/700
 - 45s - loss: 5.1084 - val_loss: 4.4068
Epoch 154/700
 - 45s - loss: 5.0872 - val_loss: 4.4424
Epoch 155/700
 - 45s - loss: 5.1035 - val_loss: 4.6599
Epoch 156/700
 - 45s - loss: 5.1577 - val_loss: 4.4092
Epoch 157/700
 - 46s - loss: 5.1429 - val_loss: 4.5303
Epoch 158/700
 - 45s - loss: 5.2774 - val_loss: 5.6741
Epoch 159/700
 - 46s - loss: 5.3746 - val_loss: 4.3145
Epoch 160/700
 - 45s - loss: 5.1669 - val_loss: 4.6520
Epoch 161/700
 - 45s - loss: 5.0976 - val_loss: 4.2803
Epoch 162/700
 - 45s - loss: 5.1596 - val_loss: 5.0436
Epoch 163/700
 - 46s - loss: 5.2355 - val_loss: 4.3119
Epoch 164/700
 - 45s - loss: 5.4332 - val_loss: 4.2752
Epoch 165/700
 - 45s - loss: 5.3822 - val_loss: 5.3062
Epoch 166/700
 - 46s - loss: 5.3523 - val_loss: 4.2783
Epoch 167/700
 - 46s - loss: 5.1947 - val_loss: 5.0290
Epoch 168/700
 - 46s - loss: 5.1119 - val_loss: 4.3011
Epoch 169/

Epoch 300/700
 - 46s - loss: 4.7820 - val_loss: 4.1882
Epoch 301/700
 - 46s - loss: 4.8339 - val_loss: 4.1980
Epoch 302/700
 - 45s - loss: 4.8966 - val_loss: 4.3713
Epoch 303/700
 - 45s - loss: 4.7946 - val_loss: 4.3067
Epoch 304/700
 - 45s - loss: 4.9059 - val_loss: 4.1581
Epoch 305/700
 - 45s - loss: 4.8047 - val_loss: 4.7546
Epoch 306/700
 - 45s - loss: 4.8597 - val_loss: 4.8552
Epoch 307/700
 - 45s - loss: 4.8990 - val_loss: 4.1732
Epoch 308/700
 - 45s - loss: 4.8889 - val_loss: 4.1986
Epoch 309/700
 - 46s - loss: 4.8216 - val_loss: 4.1652
Epoch 310/700
 - 46s - loss: 4.9021 - val_loss: 4.3303
Epoch 311/700
 - 45s - loss: 4.8911 - val_loss: 4.6850
Epoch 312/700
 - 46s - loss: 4.8048 - val_loss: 4.1692
Epoch 313/700
 - 45s - loss: 4.8105 - val_loss: 4.1819
Epoch 314/700
 - 45s - loss: 4.8300 - val_loss: 4.2016
Epoch 315/700
 - 45s - loss: 4.8082 - val_loss: 4.4571
Epoch 316/700
 - 45s - loss: 4.8536 - val_loss: 4.8918
Epoch 317/700
 - 45s - loss: 4.9194 - val_loss: 4.5853
Epoch 318/

Epoch 449/700
 - 47s - loss: 4.6237 - val_loss: 4.5369
Epoch 450/700
 - 47s - loss: 4.6660 - val_loss: 4.2312
Epoch 451/700
 - 46s - loss: 4.7665 - val_loss: 4.1295
Epoch 452/700
 - 46s - loss: 4.7135 - val_loss: 4.8235
Epoch 453/700
 - 44s - loss: 4.6851 - val_loss: 4.2270
Epoch 454/700
 - 36s - loss: 4.6274 - val_loss: 4.4376
Epoch 455/700
 - 34s - loss: 4.5974 - val_loss: 4.3617
Epoch 456/700
 - 33s - loss: 4.6043 - val_loss: 4.5492
Epoch 457/700
 - 34s - loss: 4.6161 - val_loss: 4.3172
Epoch 458/700
 - 34s - loss: 4.6176 - val_loss: 4.4855
Epoch 459/700
 - 34s - loss: 4.5956 - val_loss: 4.3754
Epoch 460/700
 - 34s - loss: 4.6533 - val_loss: 4.9506
Epoch 461/700
 - 34s - loss: 4.7595 - val_loss: 4.2513
Epoch 462/700
 - 34s - loss: 4.7622 - val_loss: 4.2169
Epoch 463/700
 - 34s - loss: 4.8404 - val_loss: 4.9631
Epoch 464/700
 - 34s - loss: 4.6925 - val_loss: 4.2506
Epoch 465/700
 - 34s - loss: 4.6364 - val_loss: 4.2320
Epoch 466/700
 - 34s - loss: 4.6830 - val_loss: 4.6677
Epoch 467/

Epoch 598/700
 - 34s - loss: 4.5476 - val_loss: 4.3891
Epoch 599/700
 - 34s - loss: 4.5049 - val_loss: 4.8032
Epoch 600/700
 - 34s - loss: 4.5124 - val_loss: 4.2403
Epoch 601/700
 - 34s - loss: 4.6446 - val_loss: 4.3118
Epoch 602/700
 - 34s - loss: 4.6518 - val_loss: 4.8174
Epoch 603/700
 - 33s - loss: 4.5419 - val_loss: 4.3197
Epoch 604/700
 - 24s - loss: 4.5395 - val_loss: 4.3936
Epoch 605/700
 - 21s - loss: 4.4280 - val_loss: 4.6133
Epoch 606/700
 - 21s - loss: 4.4555 - val_loss: 4.7454
Epoch 607/700
 - 21s - loss: 4.4552 - val_loss: 4.3933
Epoch 608/700
 - 21s - loss: 4.4054 - val_loss: 4.5141
Epoch 609/700
 - 21s - loss: 4.3972 - val_loss: 4.3694
Epoch 610/700
 - 21s - loss: 4.4142 - val_loss: 4.4104
Epoch 611/700
 - 21s - loss: 4.3983 - val_loss: 4.4918
Epoch 612/700
 - 21s - loss: 4.4312 - val_loss: 4.2791
Epoch 613/700
 - 21s - loss: 4.5185 - val_loss: 5.2871
Epoch 614/700
 - 21s - loss: 4.5778 - val_loss: 4.5403
Epoch 615/700
 - 21s - loss: 4.4142 - val_loss: 4.3891
Epoch 616/

In [25]:
ConvLSTMpred = modelConvLSTM.predict(X_valid, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))
ConvLSTMresults = metrics(ConvLSTMpred, y_valid)
ConvLSTMresults

{'Explained Variance Score': 0.6284181525356383,
 'Max Error': 38.12789489746095,
 'Mean Absolute Error': 5.675722757113182,
 'Mean Squared Error': 53.55709098902109,
 'Root Mean Squared Error': 7.318271038231714,
 'Median Absolute Error': 4.8575631713867224,
 'R² Score': 0.5933822319957598,
 'Mean Absolute Percentage Error': 5.586247492930699}

In [26]:
ConvLSTMpred = modelConvLSTM.predict(X_test, verbose=0)
ConvLSTMpred = ConvLSTMpred.reshape((ConvLSTMpred.shape[0]))


In [27]:

ConvLSTMpred 


array([ 80.17792 ,  94.32147 ,  81.20392 , ...,  94.66198 , 112.88135 ,
        94.706055], dtype=float32)